In [1]:
from typing import TypedDict, List, Literal
from langgraph.graph import StateGraph

In [3]:
# Define the state structure
class TrekState(TypedDict):
    question: str  # User's question
    answer: str    # Generated answer
    evaluation: Literal["LOGICAL", "EMOTIONAL"]  # Spock or McCoy?
    messages: List[str]  # Chat history (optional)

In [4]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [5]:
from openai import OpenAI

client = OpenAI()  # Ensure OPENAI_API_KEY is set

In [6]:
# Define the nodes
def get_question(state: TrekState):
    state["question"] = "What is the Vulcan philosophy?"
    return state

def generate_answer(state: TrekState):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[{"role": "user", "content": state["question"]}]
    )
    state["answer"] = response.choices[0].message.content
    return state

def evaluate_response(state: TrekState):
    if "spock" in state["answer"].lower() or "vulcan" in state["answer"].lower():
        state["evaluation"] = "LOGICAL"
        state["answer"] += "\n\n🖖 Live long and prosper."
    else:
        state["evaluation"] = "EMOTIONAL"
    return state
  

In [9]:
# Build the Graph
workflow = StateGraph(TrekState)

# Add nodes
workflow.add_node("get_question", get_question)
workflow.add_node("generate_answer", generate_answer)
workflow.add_node("evaluate_response", evaluate_response)

# Define edges
workflow.add_edge("get_question", "generate_answer")
workflow.add_edge("generate_answer", "evaluate_response")

# Optional: Conditional edge (if we had branching paths)
workflow.set_entry_point("get_question")
workflow.set_finish_point("evaluate_response")

# Compile
app = workflow.compile()

In [ ]:
# Initialize state
initial_state = TrekState(question="", answer="", evaluation="", messages=[])

#Execute 
result = app.invoke(initial_state)
print("Final Answer:", result["answer"])
print("Evaluation:", result["evaluation"])



Final Answer: The Vulcan philosophy is a guiding principle for the fictional Vulcan species in the Star Trek universe. It is based on the teachings of Surak, a revered Vulcan philosopher and founder of the Vulcan way of life. The Vulcan philosophy emphasizes logic, reason, self-control, and the suppression of emotions in order to achieve a harmonious and peaceful society. Vulcans believe in the pursuit of knowledge, the control of one's emotions, and the prioritization of the needs of the many over the needs of the individual.

🖖 Live long and prosper.
Evaluation: LOGICAL
